In [ ]:
#pip install pandasql 

In [ ]:
# Go to OHDSI Athena 網站下載 OHDSI vocabulary: ICD10CM and SNOMED
# https://athena.ohdsi.org/vocabulary/list

In [1]:
import pandas as pd
import pandasql as psql

# Read CSV files into a DataFrame
concept_df = pd.read_csv(r"C:\Users\wuche\Downloads\ICD10_SNOMED_CT\CONCEPT.csv", sep="\t")  # concept table

concept_rel_df = pd.read_csv(r"C:\Users\wuche\Downloads\ICD10_SNOMED_CT\concept_relationship.csv", sep="\t")  # concept_relationship table

# Run an SQL query on two DataFrames
query= """
SELECT 
    source.concept_code AS icd10_code, 
    source.concept_name AS icd10_name, 
    target.concept_code AS snomed_code, 
    target.concept_name AS snomed_name
FROM 
    concept_df AS source
INNER JOIN 
    concept_rel_df AS cr ON source.concept_id = cr.concept_id_1
INNER JOIN 
    concept_df AS target ON cr.concept_id_2 = target.concept_id
WHERE 
     source.vocabulary_id = 'ICD10CM' 
     and target.vocabulary_id = 'SNOMED'
     and cr.relationship_id = 'Maps to'
     and cr.invalid_reason IS NULL

"""
print("start")
result_df = psql.sqldf(query, locals())  # Execute SQL on DataFrame

# Display result
# for index, row in result_df.iterrows():
#     print(f"Row {index}: ICD10_CODE={row['icd10_code']}, ICD10_NAME={row['icd10_name']}, SNOMED_CODE={row['snomed_code']}, SNOMED_NAME={row['snomed_name']}")

result_df.to_csv("output.csv", index=False)  # Save without index

# Unique ICD code count
unique_icd_count = result_df['icd10_code'].nunique()

# Count total records
total_records = result_df.shape[0]

# Print results
print(f"Unique ICD-10 Codes: {unique_icd_count}")
print(f"Total Records: {total_records}")

print("Done!")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


start
Unique ICD-10 Codes: 97439
Total Records: 128055
Done!
